# Biosignal processing


#### Dependencies

In [22]:
import pandas as pd
import numpy as np
import os

In [23]:
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
import systole
import plotly
import peakdet

In [25]:
%pylab inline
sns.set_style('darkgrid')
pylab.rcParams['figure.figsize'] = (12.0, 6.0)  
plt.rcParams.update({'font.size': 14})

Populating the interactive namespace from numpy and matplotlib


/home/francois.lespinasse/anaconda3/envs/biosigEnv/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['save', 'plot', 'show']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [26]:
%run ~/cneuromod/ds_prep/physio/preproc/neuromod_process.py

In [27]:
filename = "/scratch/flesp/physio_data/friends/sub-05/ses-005/neuromod_video52020-01-28T18_16_13_06"

In [28]:
filename[filename.rfind('/')+1:]

'neuromod_video52020-01-28T18_16_13_06'

In [29]:
json=pd.read_json(f"{filename}.json")

In [30]:
fs = json['SamplingFrequency'][0]

In [31]:
bio_df = pd.read_csv(f"{filename}.tsv.gz",
                     sep='\t',
                     compression='gzip',
                     header=None,
                     names=json.Columns)

In [32]:
signals, info_corrected = neuromod_ppg_process(bio_df['PPG'], fs)

Cleaning the RR interval time series using 2 iterations.
 - Iteration 1 - 
... correcting 8 missed interval(s).
... correcting 33 ectopic interval(s).
... correcting 7 long interval(s).
 - Iteration 2 - 
... correcting 2 missed interval(s).
... correcting 10 ectopic interval(s).
... correcting 3 long interval(s).


In [33]:
plot = systole.plots.plot_raw(signals['PPG_Clean'], backend='bokeh', show_heart_rate=True, show_artefacts=True, sfreq=fs)

In [34]:
from bokeh.io import output_notebook
from bokeh.plotting import show, output_file, save
output_notebook()

show(plot)

Loading BokehJS ...

In [36]:
output_file('plot_test_sig.html')
save(plot)

'/home/francois.lespinasse/cneuromod/ds_prep/physio/notebooks/plot_test_sig.html'

In [15]:
plot_rr = systole.plots.plot_rr(info_corrected['clean_rr'],input_type='rr_ms',backend='bokeh', show_artefacts=True)

In [16]:
output_notebook()

show(plot_rr)

Loading BokehJS ...

In [ ]:
bio_df, bio_info = neuromod_bio_process(df=bio_df, sampling_rate=fs)

In [ ]:
bio_df.keys()